In [1]:
import os
from pathlib import Path

NOTEBOOK_PATH: Path = Path(os.path.abspath(''))
PATH_FOR_RESULT: Path = NOTEBOOK_PATH.parent.joinpath('RESULT')

# 获取大连商品交易所（大商所）的历史数据

[大商所官网](http://www.dce.com.cn/)顶部菜单“行情数据” -> “历史数据”。

这次信息组织的更加麻烦，不仅分年份，还分品种。

还是需要解析网页源码。

In [2]:
from typing import Dict
import datetime as dt

import requests
from lxml import etree


def fetch_dce_history_index() -> Dict[int, Dict[str, str]]:
    """
    获取大连商品交易所（大商所，DCE）的历史数据下载链接。
    :return: Dict[str, Dict[int, str]]，一个嵌套的字典。
             外层字典的 key 是期货（'futures'）或者期权（'option'），
             内层字典的 key 是年份，value 是下载地址。
    """
    # 定义返回结果。
    result: Dict[int, Dict[str, str]] = {}
    
    # 定义大商所url。
    url_dce: str = 'http://www.dce.com.cn'
    url: str = f'{url_dce}/dalianshangpin/xqsj/lssj/index.html'

    # 生成年份列表。
    year_list: List[int] = [year for year in range(2006, dt.date.today().year)]
    year_list.reverse()
    for year in year_list:
        result[year] = {}

    # 下载网页。
    response = requests.get(url)
    if response.status_code != 200:
        raise requests.exceptions.HTTPError(
            f'下载 <{url}> 时发生错误。'
        )
    response.encoding = 'utf-8'

    # 解析网页。
    html: etree._Element = etree.HTML(response.text)
    history_data_list: List[etree._Element] = html.xpath('//ul[@class="cate_sel clearfix"]')

    # 填充数据.
    for i in range(len(year_list)):
        product_list = history_data_list[i].xpath('./li/label/text()')
        url_list = history_data_list[i].xpath('./li/label/input/@rel')
        assert len(product_list) == len(url_list)
        for j in range(len(product_list)):
            result[year_list[i]][product_list[j]] = f'{url_dce}{url_list[j]}'

    return result

# 测试
print(fetch_dce_history_index())

{2020: {'玉米': 'http://www.dce.com.cn/dalianshangpin/resource/cms/2021/01/2021013022310721941.xlsx', '玉米淀粉': 'http://www.dce.com.cn/dalianshangpin/resource/cms/2021/01/2021013022303925750.xlsx', '黄大豆1号': 'http://www.dce.com.cn/dalianshangpin/resource/cms/2021/01/2021013022293775777.xlsx', '黄大豆2号': 'http://www.dce.com.cn/dalianshangpin/resource/cms/2021/01/2021013022290225068.xlsx', '豆粕': 'http://www.dce.com.cn/dalianshangpin/resource/cms/2021/01/2021013022274187325.xlsx', '豆油': 'http://www.dce.com.cn/dalianshangpin/resource/cms/2021/01/2021013022262921602.xlsx', '棕榈油': 'http://www.dce.com.cn/dalianshangpin/resource/cms/2021/01/2021013022255680433.xlsx', '纤维板': 'http://www.dce.com.cn/dalianshangpin/resource/cms/2021/01/2021013022250662794.xlsx', '胶合板': 'http://www.dce.com.cn/dalianshangpin/resource/cms/2021/01/2021013022244890750.xlsx', '鸡蛋': 'http://www.dce.com.cn/dalianshangpin/resource/cms/2021/01/2021013022243031608.xlsx', '粳米': 'http://www.dce.com.cn/dalianshangpin/resource/cms/2021

好家伙，盘满钵满的感觉。

我们还是按照年份去下载历史数据，而不是按照品种。因为我们最终都是会下载所有的品种的。

另外，以往我们面对的都是按照年份组织的数据，显得比较整齐，本地文件名都是 <交易所代码>_<年份>.zip。现在面对大商所这么散碎的链接，而且文件扩展名有 zip，有 csv，还有 xls……我们下载本地后的文件名是一个考验。

我们在以往的基础上增加品种字段，组织为：<交易所代码>_<品种>_<年份>.<扩展名>。

代码：

In [3]:
def download_dce_history_data(save_path: Path, year: int) -> None:
    """
    下载大连商品交易所（大商所，DCE）的历史数据。
    :param save_path: Path，保存的位置。
    :param year: int，需要下载数据的年份。
    :return: None。
    """
    
    # 获取下载链接。
    data_index: Dict[int, Dict[str, str]] = fetch_dce_history_index()
    
    # 声明扩展名的变量类型。
    extension_name: str
    
    # 循环下载。
    for product, url in data_index[year].items():
        # 从 url 中分离扩展名。
        extension_name = url.split('.')[-1]
        
        # 生成文件名。
        download_file = save_path.joinpath(f'DCE_{product}_{year}.{extension_name}')
        
        # 下载
        response = requests.get(url)
        
        # 如果下载不顺利，引发异常。
        if response.status_code != 200:
            raise requests.exceptions.HTTPError(
                f'下载 <{url}> 时发生错误。'
            )
        
        # 保存文件。
        with open(download_file, 'wb') as f:
            f.write(response.content)

# 测试
download_dce_history_data(save_path=PATH_FOR_RESULT, year=2020)

下载全部历史数据：

In [4]:
def download_all_dce_history_data(save_path: Path) -> None:
    """
    下载大连商品交易所（大商所，DCE）的全部历史数据。
    :param save_path: 保存的位置。
    :return: None.
    """
    start_year: int = 2006
    this_year: int = dt.date.today().year
    for year in range(start_year, this_year):
        download_dce_history_data(save_path=save_path, year=year)

# 测试
download_all_dce_history_data(save_path=PATH_FOR_RESULT)

Done.